In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
USE ROLE CRYPTO;
USE WAREHOUSE CRYPTO_DATA;
USE DATABASE CRYPTOCURRENCY;
USE SCHEMA HARMONIZED_CRYPTO;

In [ ]:
#------------------------------------------------------------------------------
# Cryptocurrency Data Engineering Pipeline
# Script:       07_update_volatility_sp.py
#------------------------------------------------------------------------------


# from snowflake.snowpark import Session
# import snowflake.snowpark.functions as F

In [ ]:
#------------------------------------------------------------------------------
# Cryptocurrency Data Engineering Pipeline
# Script:       07_update_volatility_sp.py
#------------------------------------------------------------------------------

from snowflake.snowpark import Session
import snowflake.snowpark.functions as F

def create_volatility_sp(session):
    session.use_schema('HARMONIZED_CRYPTO')
    
    sp_sql = """
    CREATE OR REPLACE PROCEDURE UPDATE_VOLATILITY_SP()
    RETURNS STRING
    LANGUAGE PYTHON
    RUNTIME_VERSION = '3.8'
    HANDLER = 'update_volatility'
    PACKAGES = ('numpy')
    AS
    $$
    import snowflake.snowpark as snowpark
    from snowflake.snowpark import functions as F
    from snowflake.snowpark.window import Window
    
    def update_volatility(session):
        # Read from harmonized table
        df = session.table("HARMONIZED_CRYPTO.CRYPTO_HARMONIZED")
        
        # Create a window spec for 10-day rolling window
        window_spec = Window.partitionBy("crypto_symbol").orderBy("date").rowsBetween(-9, 0)
        
        # Create array of closing prices over the window
        df = df.withColumn("close_array", F.collect_list("close").over(window_spec))
        
        # Apply volatility UDF
        df = df.withColumn("volatility_10d", 
                          F.call_udf("HARMONIZED_CRYPTO.CALCULATE_VOLATILITY_UDF", F.col("close_array")))
        
        # Update the harmonized table
        df = df.select("crypto_symbol", "date", "volatility_10d")
        
        # Create a temporary view for the update operation
        df.create_or_replace_temporary_view("TEMP_VOLATILITY_VIEW")
       

In [ ]:
 
        # Update the harmonized table
        update_sql = """
        UPDATE HARMONIZED_CRYPTO.CRYPTO_HARMONIZED h
        SET h.volatility_10d = t.volatility_10d
        FROM TEMP_VOLATILITY_VIEW t
        WHERE h.crypto_symbol = t.crypto_symbol
        AND h.date = t.date
        """
        session.sql(update_sql).collect()
        
        return "Volatility updated successfully in harmonized table"
    $$
    """
    session.sql(sp_sql).collect()
    
    return "Stored procedure created successfully"

# For local debugging
if __name__ == "__main__":
    # Create a local Snowpark session
    with Session.builder.getOrCreate() as session:
        result = create_volatility_sp(session)
        print(result)